### Ujian Akhir Semester<br>Kecerdasan Komputasional dan Pemebelajaran Mesin<br>
### Axl Adilla <br>20/466397/PPA/05963

In [1]:
import pandas as pd
import numpy as np

## Import Iris Dataset

In [2]:
df = pd.read_csv('iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Memeriksa Keseimbangan Data

In [3]:
df['species'].value_counts()

virginica     50
setosa        50
versicolor    50
Name: species, dtype: int64

## Memisahkan Fitur dengan Target

In [4]:
df_y = df['species'].copy()
df_x = df.drop('species', axis=1)
df_x.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## K-Fold Cross Validation

In [5]:
def custom_k_fold_validation(x, y, fold=5):
    totalData = x.shape[0]
    lengthEachFold = int(totalData/fold)
    
    categories = y.unique()
    splitCategories = []
    for i, category in enumerate(categories):
        indexes = y[y == category].index
        splitCategories.append(indexes)  
    
    kFolds = []
    for i in range(fold):
        iFolds = []
        for j in range(fold):
            indexes = []
            for l in range(lengthEachFold):
                category_id = ((j*lengthEachFold) + l) % len(categories)
                index = int(((j*lengthEachFold) + l) / len(categories))
                indexes.append(splitCategories[category_id][index])
            
            dataIfold = {
                'feature':x.loc[indexes].copy(), 
                'target':y.loc[indexes].copy()
            }
            
            iFolds.append(dataIfold)
        
        trainIndexes = [trainIndex for trainIndex in range(fold) if trainIndex != i]
        dataKfold = {
            'training' : np.array(iFolds)[trainIndexes],
            'test' : np.array(iFolds)[i]
        }
        kFolds.append(dataKfold)
    return np.array(kFolds)

In [6]:
kFold = custom_k_fold_validation(df_x, df_y, 5)
for iterasi, iFold in enumerate(kFold):
    print(f"\n\nIterasi Ke {iterasi} :")
    print(f"Training Feature :")
    for i, data in enumerate(iFold['training']):
        feature = iFold['training'][i]['feature'][:5].copy()
        target = iFold['training'][i]['target'][:5].copy()
        feature['target'] = target
        print(feature)
    print(f"\nTest Feature :")
    feature = iFold['test']['feature'][:5].copy()
    target = iFold['test']['target'][:5].copy()
    feature['target'] = target
    print(feature)



Iterasi Ke 0 :
Training Feature :
     sepal_length  sepal_width  petal_length  petal_width      target
10            5.4          3.7           1.5          0.2      setosa
60            5.0          2.0           3.5          1.0  versicolor
110           6.5          3.2           5.1          2.0   virginica
11            4.8          3.4           1.6          0.2      setosa
61            5.9          3.0           4.2          1.5  versicolor
     sepal_length  sepal_width  petal_length  petal_width      target
20            5.4          3.4           1.7          0.2      setosa
70            5.9          3.2           4.8          1.8  versicolor
120           6.9          3.2           5.7          2.3   virginica
21            5.1          3.7           1.5          0.4      setosa
71            6.1          2.8           4.0          1.3  versicolor
     sepal_length  sepal_width  petal_length  petal_width      target
30            4.8          3.1           1.6          

## Memeriksa Tiap Fold Balance

In [7]:
iFold = kFold[0]
print('Training ')
for i,fold in enumerate(iFold['training']):
    print(f'Fold Train ke-{i}')
    print(pd.DataFrame(fold['target']).value_counts())
print('\nTest ')

print(f'Fold Test')
print(pd.DataFrame(iFold['test']['target']).value_counts())

Training 
Fold Train ke-0
species   
setosa        10
versicolor    10
virginica     10
dtype: int64
Fold Train ke-1
species   
setosa        10
versicolor    10
virginica     10
dtype: int64
Fold Train ke-2
species   
setosa        10
versicolor    10
virginica     10
dtype: int64
Fold Train ke-3
species   
setosa        10
versicolor    10
virginica     10
dtype: int64

Test 
Fold Test
species   
setosa        10
versicolor    10
virginica     10
dtype: int64


## Buat Prediksi Data Iris dengan LogisticRegression dan KFold Cross Validation

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

clfs=[]
accuracies=[]
for i, iFold in enumerate(kFold):
    features = []
    for trainingIfold in iFold['training']:
        features.append(trainingIfold['feature'])
    df_feature = pd.concat(features)
    
    targets = []
    for trainingIfold in iFold['training']:
        targets.append(trainingIfold['target'])
    df_target = pd.concat(targets)
    
    encoder = LabelEncoder()
    df_target = pd.DataFrame(encoder.fit_transform(df_target.values))
    
    clf = LogisticRegression()
    clf.fit(df_feature, df_target)
    clfs.append(clf)
    
    y_test = encoder.transform(iFold['test']['target'])
    y_pred = clf.predict(iFold['test']['feature'])
    akurasi = accuracy_score(y_test, y_pred)
    print(f'Fold ke-{i}, Akurasi : {akurasi}')
    accuracies.append(akurasi)

print(f'Rata - Rata Akurasi = {np.array(accuracies).mean()}')

D:\Programs\Anaconda\envs\sent-analysis\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
D:\Programs\Anaconda\envs\sent-analysis\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Programs\Anaconda\envs\sent-analysis\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

Fold ke-0, Akurasi : 0.9666666666666667
Fold ke-1, Akurasi : 1.0
Fold ke-2, Akurasi : 0.9333333333333333
Fold ke-3, Akurasi : 0.9666666666666667
Fold ke-4, Akurasi : 1.0
Rata - Rata Akurasi = 0.9733333333333334


D:\Programs\Anaconda\envs\sent-analysis\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
